In [1]:
# Install Kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 2.3 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=11a3df83abdfa56431e363b383b1fea490c75df93ba561ba0e9ea3d89378a6ed
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
import contextlib
from google.colab import files
import io

with contextlib.redirect_stdout(io.StringIO()):
    files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c AI4Code

!  unzip /content/AI4Code.zip -d /content/data

### EDA

In [4]:
# Let's look at one json file

import pandas as pd

sample_train_df = pd.read_json('/content/data/train/ffd3afd30de9a6.json')
sample_train_df.head()

,cell_type,source
7fbecb8a,code,# This Python 3 environment comes with many he...
7142b97b,code,"import matplotlib.pyplot as plt, seaborn as sn..."
948f5a8b,code,df = pd.read_csv('/kaggle/input/spotify-top-20...
613958ba,code,# All the genres available in the data set are...
afe2bb0a,code,# Dummies are created for all genres\nnew = pd...


In [5]:
sample_train_df['cell_type'].value_counts()

code        25
markdown     1
Name: cell_type, dtype: int64

In [6]:
sample_train_df[sample_train_df['cell_type'] == 'code'].values[0]

array(['code',
       '# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session'],
      dtype=object)

In [7]:
sample_train_df[sample_train_df['cell_type'] == 'markdown'].values[0]

array(['markdown',
       '## Cluster 3 contains the set of music which are most popular and are most recent'],
      dtype=object)

In [8]:
# Training orders

train_orders_df = pd.read_csv('/content/data/train_orders.csv')
train_orders_df.head()

,id,cell_order
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 7...
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 5...
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 0...
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 d...
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a...


In [9]:
# Ancestors

train_ancestors_df = pd.read_csv('/content/data/train_ancestors.csv')
train_ancestors_df.head()

,id,ancestor_id,parent_id
0,00001756c60be8,945aea18,NaN
1,00015c83e2717b,aa2da37e,317b65d12af9df
2,0001bdd4021779,a7711fde,NaN
3,0001daf4c2c76d,090152ca,NaN
4,0002115f48f982,272b483a,NaN


In [10]:
# Filter training orders and ancestors data for a sample file

train_orders_df[train_orders_df['id'] == 'ffd3afd30de9a6']

,id,cell_order
139160,ffd3afd30de9a6,7fbecb8a 7142b97b 948f5a8b 613958ba afe2bb0a d...


In [11]:
train_ancestors_df[train_ancestors_df['id'] == 'ffd3afd30de9a6']

,id,ancestor_id,parent_id
139160,ffd3afd30de9a6,d131f56b,NaN


In [12]:
# Is this ancestor ID present in the data?
train_ancestors_df[train_ancestors_df['id'] == 'd131f56b']

,id,ancestor_id,parent_id


In [13]:
# And a sample file from testing folder
sample_test_df = pd.read_json('/content/data/test/0009d135ece78d.json')
sample_test_df.head()

,cell_type,source
ddfd239c,code,import numpy as np # linear algebra\nimport pa...
c6cd22db,code,df = pd.read_csv('/kaggle/input/breast-cancer-...
1372ae9b,code,"numerical_data = df.loc[:, ~df.columns.isin(['..."
90ed07ab,code,"def comparison_plot_maker(data_1, data_2, name..."
7f388a41,code,# Ploting data with different columns\n#######...


### Understanding Kendall Tau Metric

We can compute the Kendall tau correlation between the predicted cell orders and the ground truth cell orders by counting how many swaps of adjacent cells are needed to sort the predicted order into the ground truth order.

In [14]:
from bisect import bisect


# Actually O(N^2), but fast in practice for our data
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):  # O(N)
        j = bisect(sorted_so_far, u)  # O(log N)
        inversions += i - j
        sorted_so_far.insert(j, u)  # O(N)
    return inversions

def kendall_tau(ground_truth, predictions):
    total_inversions = 0  # total inversions in predicted ranks across all instances
    total_2max = 0  # maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max


t1 = [['a', 'b', 'c', 'd']]
t2 = [['a', 'c', 'b', 'd']]

kendall_tau(t1, t2)

0.6666666666666667

In [15]:
# KT on 2 completely opposite lists

t3 = [[i for i in range(1, 100)]]
t4 = [[j for j in range(99, 0, -1)]]

kendall_tau(t3, t4)

-1.0

In [16]:
# KT on the list itself?
kendall_tau(t3, t3)

1.0

Ideally during the training loops, we want the Kendall Tau Correlation between input and the prediction to be as close to 1 as possible

In [25]:
# Read random 100 training files under 1 dataframe

n = 10
train_dir = '/content/data/train'


import random, os, json
a = os.listdir(train_dir)
random.shuffle(a)

training_files = [os.path.join(train_dir, i) for i in a[:n]]


for i in range(n):
  if i == 0:
    small_train_df = pd.read_json(training_files[i])

  else:
    small_train_df = small_train_df.append(pd.read_json(training_files[i]))

small_train_df.head()

,cell_type,source
b6ac89ad,code,# importing libraries \nimport pandas as pd\ni...
655bf587,code,df = pd.read_csv('../input/eda-attacks-on-poli...
7cc13663,code,df.head(10)
213e442e,code,df.info()
4ebb2f83,code,df.shape
